# Basic MCP

ModelContextProtocol (MCP) allows to easily extend application and LLM capabilities using standardized feature implementation. draive library comes with support for MCP both as a server and client allowing to build LLM based application even faster with more code reuse. Lets have a small example:

In [1]:
from draive import (
    ConversationMessage,
    Toolbox,
    conversation_completion,
    ctx,
    load_env,
    setup_logging,
)
from draive.mcp import MCPClient
from draive.openai import OpenAIChatConfig, openai_lmm

load_env() # load .env variables
setup_logging("mcp")


# initialize dependencies and configuration
async with ctx.scope(
    "mcp",
    openai_lmm(),  # define used LMM to use OpenAI
    OpenAIChatConfig(model="gpt-4o-mini"),  # configure OpenAI model
    # prepare MCPClient, it will handle connection lifetime through context
    # and provide associated state with MCP functionalities
    disposables=[
        # we are going to use stdio connection with one of the example servers
        MCPClient.stdio(
            command="npx",
            args=[
                "-y",
                "@modelcontextprotocol/server-filesystem",
                "/Users/miquido/Desktop",
            ],
        ),
    ]
):
    # request model using any appropriate method, i.e. conversation for chat
    response: ConversationMessage = await conversation_completion(
        # provide a prompt instruction
        instruction="You can access files on behalf of the user on their machine using available tools."
        " Desktop directory path is `/Users/miquido/Desktop`",
        # add user input
        input="What is on my desktop?",
        # define tools available to the model from MCP extensions
        tools=await Toolbox.external(),
    )
    print(response.content)

15/Jan/2025:17:14:39 +0000 [DEBUG] [mcp] [86261f3e9c354cf1bc0cf4ef2101076a] [mcp] [d238c2ce05264b3fa7c721fdfe0d201f] Entering context...
15/Jan/2025:17:14:40 +0000 [DEBUG] [conversation_completion] [86261f3e9c354cf1bc0cf4ef2101076a] [conversation_completion] [d3668c8ed9d2464cab020a3f3f0b6020] Entering context...
15/Jan/2025:17:14:40 +0000 [DEBUG] [openai_lmm_invocation] [86261f3e9c354cf1bc0cf4ef2101076a] [openai_lmm_invocation] [69c8fbb15c26420292d3ed126c6919bd] Entering context...
15/Jan/2025:17:14:40 +0000 [DEBUG] [httpcore.connection] connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
15/Jan/2025:17:14:40 +0000 [DEBUG] [httpcore.connection] connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x14a258b00>
15/Jan/2025:17:14:40 +0000 [DEBUG] [httpcore.connection] start_tls.started ssl_context=<ssl.SSLContext object at 0x149c7c4d0> server_hostname='api.openai.com' timeout=5.0
15/Jan/2025:17:14:40 +0000 [DEB